# 数据格式化

In [443]:
import numpy as np
data = {'C': 'max 1 2 4', 'A': '2 1 3 =\n3 1 4 =', 'B': '20 25','xRange': '[0,e)\n[0,e)\n[2,6]'}


global maxNum 
maxNum = 9999999999

def formatData(data):
    # 目标函数处理
    C = data['C'].split(' ')

    questionType_f = C[0]
    C_f = np.array(C[1:]).astype(float)
    
    # 约束条件处理
    symbolMatrix_f = np.array([])     # 符号矩阵
    A_f = np.array([])              # 系数矩阵

    for i in data['A'].split('\n'):
        i = i.split(' ')
        symbolMatrix_f = np.append(symbolMatrix_f,i[-1])
        if A_f.size>0:
            A_f = np.vstack([A_f,i[:-1]])
        else:
            A_f = np.array(i[:-1])
    A_f = A_f.astype(float)

    # 取值范围处理
    xRange = data['xRange']

    xRange_f = np.array([eval(i) for i in xRange.replace('(','[').\
                                replace(')',']').\
                                replace('e',str(maxNum)).\
                                strip('\n').
                                split('\n')
            ])

    # 右端向量处理
    B = data['B']
    B_f = np.array(B.split(' ')).astype(float)

    data_f={
        'questionType':questionType_f,
        'C':C_f,
        'A':A_f,
        'symbolMatrix':symbolMatrix_f,
        'B':B_f,
        'xRange':xRange_f,
        }

    return data_f
    
    

global fData
fData = formatData(data) 

fData

{'questionType': 'max',
 'C': array([1., 2., 4.]),
 'A': array([[2., 1., 3.],
        [3., 1., 4.]]),
 'symbolMatrix': array(['=', '='], dtype='<U32'),
 'B': array([20., 25.]),
 'xRange': array([[         0, 9999999999],
        [         0, 9999999999],
        [         2,          6]], dtype=int64)}

# X变量名注册、删除、插入

In [444]:
global xName
xName= []

def regX(s):
    global xName
    xName.append(s)

def delX(s):
    try:
        index = xName.index(s)
        xName.remove(s)
        return index
    except:
        pass

def insertX(s,index):
    xName.insert(index,s)

def getXname(index):
    return 'x{}'.format(index+1)

def getXPname(index):
    return 'xp{}'.format(index+1)

def generateXname():
    global xName
    xName=[]
    for i in range(fData['A'][0].size):
        regX('x{}'.format(str(i+1)))

generateXname()

xName

['x1', 'x2', 'x3']

# 数据标准化

In [445]:
'''
{'questionType': 'min',
 'C': array(['-1', '-1'], dtype='<U2'),
 'A': array([['1', '2', '1', '0'],
        ['3', '5', '0', '1']], dtype='<U1'),
 'symbolMatrix': array(['=', '>'], dtype='<U32'),
 'B': array(['4', '10'], dtype='<U2'),
 'xRang': array([[         0, 9999999999],
        [         0, 9999999999],
        [         0, 9999999999],
        [         0, 9999999999]], dtype=int64)}
'''

def standard(data):
    print('原数据：')
    print(data)

    # 检查是哪一类
    erroTypeList = checkErroType(data)
    print('错误类型有：'+str(erroTypeList))

    # 处理决策变量范围不对
    if '3' in erroTypeList:
        data = handleXrange(data)

    # 处理限额系数为负数
    if '4' in erroTypeList:
        data = handleB(data)

    # 处理约束为不等式
    if '2' in erroTypeList:
        data = handlesymbolMatrix(data)

    # # 处理目标函数极大化
    if '1' in erroTypeList:
        data = handleC(data)

    print('修改后')
    print(data)
        


def checkErroType(data):
    erroTypeList = []   

    # 1.目标函数极大化
    if data['questionType'] != 'min':
        erroTypeList.append('1')

    # 2.约束为不等式
    if np.sum(data['symbolMatrix']=='=') != data['symbolMatrix'].size:
        erroTypeList.append('2')

    # 3.决策变量范围不对
    if np.sum(data['xRange']==[0,maxNum]) != data['xRange'].size:
        erroTypeList.append('3')

    # 4.限额系数为负数
    if np.sum(data['B'].astype(float)>=0) != data['B'].size:
        erroTypeList.append('4')

        
    return erroTypeList

# 处理决策变量范围不对
def handleXrange(data):
    saveXrange = data['xRange']

    # 偏移量
    offset = 0
    for (index,i) in enumerate(saveXrange):
        offIndex = offset+index

        # 小于0
        if (i ==[-maxNum,0]).all():
            # 变量名处理
            # xIndex = delX(getXname(index))
            # insertX(getXPname(index),xIndex)

            # 矩阵处理
            data['xRange'] = np.delete(data['xRange'],offIndex,axis=0)
            data['xRange'] = np.insert(data['xRange'],offIndex,[0,maxNum],axis=0)

            data['A'][:,offIndex] = np.dot(-1,data['A'][:,offIndex])

            data['C'][offIndex] = data['C'][offIndex]*-1
            

        # 无限制
        if (i == [-maxNum,maxNum]).all():
            data['xRange'] = np.delete(data['xRange'],offIndex,axis=0)
            data['xRange'] = np.insert(data['xRange'],offIndex,[0,maxNum],axis=0)
            data['xRange'] = np.insert(data['xRange'],offIndex,[0,maxNum],axis=0)

            c = data['A'][:,offIndex]*-1
            data['A'] = np.insert(data['A'],offIndex+1,c,axis=1)
            
            c = data['C'][offIndex]*np.array([1,-1])
            data['C'] = np.delete(data['C'],offIndex)
            data['C'] = np.insert(data['C'],offIndex,c)

            offset+=1

        # 小于常数
        if (i[0]==-maxNum and i[1]!=maxNum and i[1]!=0).all():

            # 常数项改变            
            data['B'] = data['B']-data['A'][:,offIndex]*data['xRange'][offIndex][1]

            # 变量变为相反数
            data['xRange'] = np.delete(data['xRange'],offIndex,axis=0)
            data['xRange'] = np.insert(data['xRange'],offIndex,[0,maxNum],axis=0)

            data['A'][:,offIndex] = -1 * data['A'][:,offIndex]

            data['C'][offIndex] = -1 * data['C'][offIndex]


        # 范围
        if (i[0]!=-maxNum and i[1]!=maxNum).all():

            
            # 常数项改变
            data['B'] = data['B']-data['A'][:,offIndex]*data['xRange'][offIndex][0]

            # 新增条件
            row = np.zeros(data['A'][0].size)
            row[-1] = 1
            data['A'] = np.insert(data['A'],data['A'][:,0].size,row,axis=0)
            
            col = np.zeros(data['A'][:,0].size)
            col[-1] = 1
            data['A']= np.insert(data['A'],data['A'][0].size,col,axis=1)

            data['B'] = np.append(data['B'],data['xRange'][offIndex][1]-data['xRange'][offIndex][0])

            data['C'][offIndex] = data['C'][offIndex]*-1
            data['C'] = np.append(data['C'],0)

            data['xRange'] = np.delete(data['xRange'],offIndex,axis=0)
            data['xRange'] = np.insert(data['xRange'],offIndex,[0,maxNum],axis=0)
            data['xRange'] = np.insert(data['xRange'],offIndex,[0,maxNum],axis=0)

            offset+=1
            
    return data

# 处理限额系数为负数
def handleB(data):
    for (index,i) in enumerate(data['B']):
        if i <0:
            data['A'][index] = data['A'][index]*-1
            data['B'][index] = data['B'][index]*-1
    
    return  data

# 处理约束为不等式
def handlesymbolMatrix(data):
    for (index,i) in enumerate(data['symbolMatrix']):

        # 大于约束，减去松弛变量
        if '>' in i:
            sc=-1

        # 小于约束，增加松弛变量
        elif '<' in i:
            sc=1
        else:
            continue
        
        data['xRange'] = np.append(data['xRange'],np.array([0,maxNum]).reshape(1,2),axis=0)

        c = np.zeros(data['A'][:,0].size)
        c[index] = sc

        data['A'] = np.append(data['A'],c.reshape(data['A'][:,0].size,1),axis=1)
        data['C'] = np.append(data['C'],0)

        data['symbolMatrix'] = '='


    return data



# 处理目标函数极大化
def handleC(data):
    data['questionType'] = 'min'
    data['C'] = data['C']*-1

    return data


global stdData
stdData = standard(fData)




原数据：
{'questionType': 'max', 'C': array([1., 2., 4.]), 'A': array([[2., 1., 3.],
       [3., 1., 4.]]), 'symbolMatrix': array(['=', '='], dtype='<U32'), 'B': array([20., 25.]), 'xRange': array([[         0, 9999999999],
       [         0, 9999999999],
       [         2,          6]], dtype=int64)}
错误类型有：['1', '3']
修改后
{'questionType': 'min', 'C': array([-1., -2.,  4.]), 'A': array([[2., 1., 3., 0.],
       [3., 1., 4., 0.],
       [0., 0., 1., 1.]]), 'symbolMatrix': array(['=', '='], dtype='<U32'), 'B': array([14., 17.,  4.]), 'xRange': array([[         0, 9999999999],
       [         0, 9999999999],
       [         0, 9999999999],
       [         0, 9999999999]], dtype=int64)}


# 求解线性规划问题

In [446]:
def getAns(data):
    pass

getAns(stdData)

In [447]:
a=[[1,2],[3,4]]

a = np.array(a)

a[:,0] = -1 * a[:,0]

a

array([[-1,  2],
       [-3,  4]])